## Extensions and discussions

Firstly, we will import the required libraries and load the `Advertising` dataset.

In [ ]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api import ols

%matplotlib inline
plt.style.use("seaborn-white")

**Load Datasets**

Datasets available on https://www.statlearning.com/resources-first-edition

In [ ]:
credit_url = "https://github.com/pykale/transparentML/raw/main/data/Credit.csv"

credit_df = pd.read_csv(credit_url)
credit_df["Student2"] = credit_df.Student.map({"No": 0, "Yes": 1})
credit_df.head(3)

In [ ]:
auto_url = "https://github.com/pykale/transparentML/raw/main/data/Auto.csv"

auto_df = pd.read_csv(auto_url, na_values="?").dropna()
auto_df.info()

### Figure 3.6

In [ ]:
sns.pairplot(
    credit_df[["Balance", "Age", "Cards", "Education", "Income", "Limit", "Rating"]]
)

###  Table 3.7

In [ ]:
est = ols("Balance ~ Own", credit_df).fit()
est.summary().tables[1]

### Table 3.8

In [ ]:
est = ols("Balance ~ Ethnicity", credit_df).fit()
est.summary().tables[1]

### Table 3.9 - Interaction Variables

In [ ]:
est = ols("Sales ~ TV + Radio + TV*Radio", advertising_df).fit()
est.summary().tables[1]

### Figure 3.7 - Interaction between qualitative and quantative variables

In [ ]:
est1 = ols("Balance ~ Income + Student2", credit_df).fit()
regr1 = est1.params
est2 = ols("Balance ~ Income + Income*Student2", credit_df).fit()
regr2 = est2.params

print("Regression 1 - without interaction term")
print(regr1)
print("\nRegression 2 - with interaction term")
print(regr2)

In [ ]:
# Income (x-axis)
income = np.linspace(0, 150)

# Balance without interaction term (y-axis)
student1 = np.linspace(
    regr1["Intercept"] + regr1["Student2"],
    regr1["Intercept"] + regr1["Student2"] + 150 * regr1["Income"],
)
non_student1 = np.linspace(
    regr1["Intercept"], regr1["Intercept"] + 150 * regr1["Income"]
)

# Balance with iteraction term (y-axis)
student2 = np.linspace(
    regr2["Intercept"] + regr2["Student2"],
    regr2["Intercept"]
    + regr2["Student2"]
    + 150 * (regr2["Income"] + regr2["Income:Student2"]),
)
non_student2 = np.linspace(
    regr2["Intercept"], regr2["Intercept"] + 150 * regr2["Income"]
)

# Create plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(income, student1, "r", income, non_student1, "k")
ax2.plot(income, student2, "r", income, non_student2, "k")

for ax in fig.axes:
    ax.legend(["student", "non-student"], loc=2)
    ax.set_xlabel("Income")
    ax.set_ylabel("Balance")
    ax.set_ylim(ymax=1550)

### Figure 3.8 - Non-linear relationships

In [ ]:
# With Seaborn's regplot() you can easily plot higher order polynomials.
plt.scatter(
    auto_df.horsepower, auto_df.mpg, facecolors="None", edgecolors="k", alpha=0.5
)
sns.regplot(
    auto_df.horsepower,
    auto_df.mpg,
    ci=None,
    label="Linear",
    scatter=False,
    color="orange",
)
sns.regplot(
    auto_df.horsepower,
    auto_df.mpg,
    ci=None,
    label="Degree 2",
    order=2,
    scatter=False,
    color="lightblue",
)
sns.regplot(
    auto_df.horsepower,
    auto_df.mpg,
    ci=None,
    label="Degree 5",
    order=5,
    scatter=False,
    color="g",
)
plt.legend()
plt.ylim(5, 55)
plt.xlim(40, 240);

### Table 3.10

In [ ]:
auto_df["horsepower2"] = auto_df.horsepower**2
auto_df.head(3)

In [ ]:
est = ols("mpg ~ horsepower + horsepower2", auto_df).fit()
est.summary().tables[1]

### Figure 3.9

In [ ]:
regr = LinearRegression()

# Linear fit
X = auto_df.horsepower.values.reshape(-1, 1)
y = auto_df.mpg
regr.fit(X, y)

auto_df["pred1"] = regr.predict(X)
auto_df["resid1"] = auto_df.mpg - auto_df.pred1

# Quadratic fit
X2 = auto_df[["horsepower", "horsepower2"]].as_matrix()
regr.fit(X2, y)

auto_df["pred2"] = regr.predict(X2)
auto_df["resid2"] = auto_df.mpg - auto_df.pred2

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
sns.regplot(
    auto_df.pred1,
    auto_df.resid1,
    lowess=True,
    ax=ax1,
    line_kws={"color": "r", "lw": 1},
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax1.hlines(
    0,
    xmin=ax1.xaxis.get_data_interval()[0],
    xmax=ax1.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax1.set_title("Residual Plot for Linear Fit")

# Right plot
sns.regplot(
    auto_df.pred2,
    auto_df.resid2,
    lowess=True,
    line_kws={"color": "r", "lw": 1},
    ax=ax2,
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax2.hlines(
    0,
    xmin=ax2.xaxis.get_data_interval()[0],
    xmax=ax2.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax2.set_title("Residual Plot for Quadratic Fit")

for ax in fig.axes:
    ax.set_xlabel("Fitted values")
    ax.set_ylabel("Residuals")

### Figure 3.14

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
ax1.scatter(credit_df.Limit, credit_df.Age, facecolor="None", edgecolor="r")
ax1.set_ylabel("Age")

# Right plot
ax2.scatter(credit_df.Limit, credit_df.Rating, facecolor="None", edgecolor="r")
ax2.set_ylabel("Rating")

for ax in fig.axes:
    ax.set_xlabel("Limit")
    ax.set_xticks([2000, 4000, 6000, 8000, 12000])

### Figure 3.15

In [ ]:
y = credit_df.Balance

# Regression for left plot
X = credit_df[["Age", "Limit"]].as_matrix()
regr1 = LinearRegression()
regr1.fit(scale(X.astype("float"), with_std=False), y)
print("Age/Limit\n", regr1.intercept_)
print(regr1.coef_)

# Regression for right plot
X2 = credit_df[["Rating", "Limit"]].as_matrix()
regr2 = LinearRegression()
regr2.fit(scale(X2.astype("float"), with_std=False), y)
print("\nRating/Limit\n", regr2.intercept_)
print(regr2.coef_)

In [ ]:
# Create grid coordinates for plotting
B_Age = np.linspace(regr1.coef_[0] - 3, regr1.coef_[0] + 3, 100)
B_Limit = np.linspace(regr1.coef_[1] - 0.02, regr1.coef_[1] + 0.02, 100)

B_Rating = np.linspace(regr2.coef_[0] - 3, regr2.coef_[0] + 3, 100)
B_Limit2 = np.linspace(regr2.coef_[1] - 0.2, regr2.coef_[1] + 0.2, 100)

X1, Y1 = np.meshgrid(B_Limit, B_Age, indexing="xy")
X2, Y2 = np.meshgrid(B_Limit2, B_Rating, indexing="xy")
Z1 = np.zeros((B_Age.size, B_Limit.size))
Z2 = np.zeros((B_Rating.size, B_Limit2.size))

Limit_scaled = scale(credit_df.Limit.astype("float"), with_std=False)
Age_scaled = scale(credit_df.Age.astype("float"), with_std=False)
Rating_scaled = scale(credit_df.Rating.astype("float"), with_std=False)

# Calculate Z-values (RSS) based on grid of coefficients
for (i, j), v in np.ndenumerate(Z1):
    Z1[i, j] = (
        (y - (regr1.intercept_ + X1[i, j] * Limit_scaled + Y1[i, j] * Age_scaled)) ** 2
    ).sum() / 1000000

for (i, j), v in np.ndenumerate(Z2):
    Z2[i, j] = (
        (y - (regr2.intercept_ + X2[i, j] * Limit_scaled + Y2[i, j] * Rating_scaled))
        ** 2
    ).sum() / 1000000

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.suptitle("RSS - Regression coefficients", fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

min_RSS = r"$\beta_0$, $\beta_1$ for minimized RSS"

# Left plot
CS = ax1.contour(X1, Y1, Z1, cmap=plt.cm.Set1, levels=[21.25, 21.5, 21.8])
ax1.scatter(regr1.coef_[1], regr1.coef_[0], c="r", label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")
ax1.set_ylabel(r"$\beta_{Age}$", fontsize=17)

# Right plot
CS = ax2.contour(X2, Y2, Z2, cmap=plt.cm.Set1, levels=[21.5, 21.8])
ax2.scatter(regr2.coef_[1], regr2.coef_[0], c="r", label=min_RSS)
ax2.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")
ax2.set_ylabel(r"$\beta_{Rating}$", fontsize=17)
ax2.set_xticks([-0.1, 0, 0.1, 0.2])

for ax in fig.axes:
    ax.set_xlabel(r"$\beta_{Limit}$", fontsize=17)
    ax.legend()

### Variance Inflation Factor - page 102

In [ ]:
est_Age = ols("Age ~ Rating + Limit", credit_df).fit()
est_Rating = ols("Rating ~ Age + Limit", credit_df).fit()
est_Limit = ols("Limit ~ Age + Rating", credit_df).fit()

print(1 / (1 - est_Age.rsquared))
print(1 / (1 - est_Rating.rsquared))
print(1 / (1 - est_Limit.rsquared))